# 📓 08_transform_playlist_items_snapshot.ipynb

Este notebook es 100% modelado, no extracción.

🎯 Objetivo

Crear el histórico mensual de la composición de playlists manuales.

1 fila = 1 video en 1 playlist en 1 mes.

Este notebook:

❌ NO llama a la API  
❌ NO reescribe estado  
✅ Agrega contexto temporal  
✅ Permite histórico

In [ ]:
from pathlib import Path
import pandas as pd
from datetime import datetime, date
import requests
import os
from dotenv import load_dotenv

load_dotenv()

PROJECT_ROOT = Path.cwd().parent
PROCESSED_PATH = PROJECT_ROOT / "data" / "processed" / "youtube"
RAW_PATH = PROJECT_ROOT / "data" / "raw" / "youtube"

API_KEY = os.getenv("YOUTUBE_API_KEY")


In [ ]:
df_playlists = pd.read_parquet(
    PROCESSED_PATH / "playlists_manual_static.parquet"
)

df_playlists[["playlist_id", "title"]].head()


In [ ]:
from datetime import date, datetime, timezone

# Fecha de snapshot (solo fecha, sin hora)
SNAPSHOT_DATE = date.today()

# Timestamp explícitamente en UTC (timezone-aware)
EXTRACTED_AT = datetime.now(timezone.utc)

SNAPSHOT_DATE, EXTRACTED_AT


⚠️ Nota (Arquitectura Final)

En esta versión del notebook, el playlist_items_snapshot se construye realizando llamadas directas a la API de YouTube para cada playlist.

Sin embargo, en la versión final del pipeline en producción:

Primero se actualiza playlist_items_manual_static.

Luego playlist_items_snapshot se construye a partir de esa tabla static.

In [ ]:
rows = []

for _, playlist in df_playlists.iterrows():
    playlist_id = playlist["playlist_id"]

    page_token = None

    while True:
        params = {
            "part": "snippet,contentDetails",
            "playlistId": playlist_id,
            "maxResults": 50,
            "key": API_KEY
        }

        if page_token:
            params["pageToken"] = page_token

        response = requests.get(
            "https://www.googleapis.com/youtube/v3/playlistItems",
            params=params
        )
        response.raise_for_status()

        data = response.json()

        for item in data.get("items", []):
            rows.append({
                "snapshot_date": SNAPSHOT_DATE,
                "playlist_id": playlist_id,
                "video_id": item["contentDetails"]["videoId"],
                "position": item["snippet"]["position"],
                "added_at": item["snippet"]["publishedAt"],
                "extracted_at": EXTRACTED_AT
            })

        page_token = data.get("nextPageToken")
        if not page_token:
            break


In [ ]:
df_playlist_items_snapshot = pd.DataFrame(rows)

df_playlist_items_snapshot.head()


In [ ]:
print("Filas:", len(df_playlist_items_snapshot))
print("Playlists únicas:", df_playlist_items_snapshot["playlist_id"].nunique())
print("Videos únicos:", df_playlist_items_snapshot["video_id"].nunique())


In [ ]:
df_playlist_items_snapshot.dtypes

In [ ]:
# added_at → datetime UTC
df_playlist_items_snapshot["added_at"] = (
    pd.to_datetime(df_playlist_items_snapshot["added_at"], utc=True)
)

In [ ]:

df_playlist_items_snapshot.dtypes

In [ ]:
df_playlist_items_snapshot = df_playlist_items_snapshot[
    [
        "snapshot_date",
        "playlist_id",
        "video_id",
        "position",
        "added_at",
        "extracted_at",
    ]
]


> ⚠️ Nota (fase de desarrollo)
>
> El guardado en formato Parquet se utiliza temporalmente para pruebas y separación entre notebooks.
> En la versión final del pipeline (.py), este paso será omitido y el DataFrame se enviará directamente a BigQuery, donde residirá el histórico definitivo.


In [ ]:
output_file = PROCESSED_PATH / "playlist_items_snapshot.parquet"

df_playlist_items_snapshot.to_parquet(output_file, index=False)

output_file
